In [1]:
import numpy as np
import cv2
from filter import Kalman
import matplotlib.pyplot as plt

In [2]:
class Kalman():
    def __init__(self,dt,U,std):

        #dt: time intravel in which readings taken
        #U: acceleration in x and y direction [ux,uy] as list
        #std : variance in x direction,y direction, and variance in acceleration as list [std_x,std_y,std_acc]


        # initialize [[px,py,vx,vy]] as zero
        self.X = np.zeros(shape=(4,1))
        self.U = np.array(U).reshape(2,-1)
        self.dt = dt
        # error in measurements x and y (ie, std deviation of measurements)
        self.xm_std,self.ym_std,self.std_acc = std[0],std[1],std[2]

        # Define the State Transition Matrix A
        self.A = np.array([[1, 0, self.dt, 0],
                            [0, 1, 0, self.dt],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])
        # input control matrix
        self.B = np.array([[(self.dt**2)/2, 0],
                            [0, (self.dt**2)/2],
                            [self.dt,0],
                            [0,self.dt]])

        # since we are tracking only position of a moving object we have (we are not tracking velocity)
        self.H = np.array([[1,0,0,0],[0,1,0,0]])

        #process covariance matric  # for now we initialize as an identity matrix
        self.P = np.eye(self.A.shape[0])

        #process noise covariance matrix  // Dynamic noise
        #standard deviation of position as the standard deviation of acceleration multiplied by dt**2/2
        self.Q = np.array([[(self.dt**4)/4, 0, (self.dt**3)/2, 0],
                            [0, (self.dt**4)/4, 0, (self.dt**3)/2],
                            [(self.dt**3)/2, 0, self.dt**2, 0],
                            [0, (self.dt**3)/2, 0, self.dt**2]]) * self.std_acc**2

        #measurement noise covariance matrix  // Measurement noise
        self.R = np.array([[self.xm_std**2,0],
                           [0, self.ym_std**2]])

        self.process_noise = 0
        self.measurement_noise = 0

        
    def predict(self):
        
        # predict the distance and velocity
        self.X = np.dot(self.A,self.X) + np.dot(self.B,self.U) + self.process_noise

        # predicted process cov matrix
        self.P = np.dot(np.dot(self.A,self.P),self.A.T) + self.Q
        
        return self.X[0:2]

    def update(self,Xm):
        Xm = np.array(Xm).reshape(2,1)
        
        # calculate kalaman gain
        # K = P * H'* inv(H*P*H'+R)
        denominator = np.dot(self.H,np.dot(self.P,self.H.T)) + self.R
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(denominator)) #shape: (4,2)
        

        # measurments
        C = np.eye(Xm.shape[0])
        Xm = np.dot(C,Xm) + self.measurement_noise

        # update the predicted_state to get final prediction of iteration and process_cov_matrix
        self.X = self.X + np.dot(K,(Xm - np.dot(self.H,self.X)))

        #update process cov matrix
        self.P = (np.eye(K.shape[0]) - np.dot(np.dot(K,self.H),self.P))



        return self.X[0:2]

In [3]:
def detect_object(img):
    # convert to grayscale
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # detecting ball using canny edge detector
    edges = cv2.Canny(img,30,200)
    
    # thresolding image to two values. Those values above 254 is given as 255 and rest 0
    ret, edges = cv2.threshold(edges,254,255,cv2.THRESH_BINARY)
    
    # detect countors from this image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Set the accepted minimum & maximum radius of a detected object
    min_radius_thresh= 3
    max_radius_thresh= 100   
    centers=[]
    for c in contours:
        # ref: https://docs.opencv.org/trunk/dd/d49/tutorial_py_contour_features.html
        (x, y), radius = cv2.minEnclosingCircle(c)
        radius = int(radius)
        #Take only the valid circles
        if (radius > min_radius_thresh) and (radius < max_radius_thresh):
            centers.append(np.array([[x], [y]]))
            
    return centers

In [4]:
KF = Kalman(dt=0.1, U=[1, 1], std=[0.1,0.1,1])
VideoCap = cv2.VideoCapture('ball.mp4')

while(True):
    # Read frame
    ret, frame = VideoCap.read()
    centers = detect_object(frame)
    # If centroids are detected then track them
    if (len(centers) > 0):
        # Draw the detected circle
        cv2.circle(frame, (int(centers[0][0]), int(centers[0][1])), 27, (0, 191, 255), 2)
        
        #predict the object
        (x,y) = KF.predict()
        
        cv2.rectangle(frame, (x - 30, y - 30), (x + 30, y + 30), (255, 0, 0), 2)
        # Update
        (x1, y1) = KF.update(centers[0])
        # Draw a rectangle as the estimated object position
        cv2.rectangle(frame, (x1 - 30, y1 - 30), (x1 + 30, y1 + 30), (0, 0, 255), 2)
        
        cv2.putText(frame, "Final Estimated Position", (x1 + 15, y1 + 10), 0, 0.5, (0, 0, 255), 2)
        cv2.putText(frame, "Predicted Position", (x + 15, y), 0, 0.5, (255, 0, 0), 2)
        cv2.putText(frame, "Measured Position", (centers[0][0] + 15, centers[0][1] - 15), 0, 0.5, (0,191,255), 2)
        
        cv2.imshow('image', frame)
        if cv2.waitKey(2) & 0xFF == ord('q'):
            VideoCap.release()
            cv2.destroyAllWindows()
            break    
            
        cv2.waitKey(50)

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
